In [89]:
import pretty_midi
from mido import MidiFile, MidiTrack, Message as MidiMessage
import numpy as np

midi_data = pretty_midi.PrettyMIDI('aria_melody.mid')
sample_interval = 200
interval_ms = sample_interval*0.00002
#print(midi_data.estimate_tempo())

piano_roll = midi_data.get_piano_roll(fs=sample_interval, times=None)
piano_roll.shape
#piano_roll[0:5]
#midi_data.write('pretty_test1.mid')

(128, 33840)

In [41]:
#velocity=100, pitch=note_number, start=0, end=.5)
np.set_printoptions(threshold=np.nan)
#FORMAT OF PIANO ROLL: piano_roll[note,time]
print(piano_roll[81,:])

[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0. 

In [90]:
# Create a PrettyMIDI object
midi_obj = pretty_midi.PrettyMIDI()
# Create an Instrument instance for a cello instrument
piano_program = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
piano = pretty_midi.Instrument(program=piano_program)


time_steps = piano_roll.shape[1]
#piano_roll2 = piano_roll[~np.all(piano_roll == 0, axis=1)]
t = 0
offset = 0.0
n_notes = piano_roll.shape[0]
while(t < time_steps):
    #Search for highest note which is activated
    k = n_notes - 1
    while( k >= 0):
        if(piano_roll[k,t] != 0):
            t0 = t # Storing the start position of the note
            while(piano_roll[k,t] != 0):
                #NOTE ON
                t = t + 1
                if(t >= time_steps):
                    break
            # Insert note
            note = pretty_midi.Note(velocity=100, pitch=k, start=t0*interval_ms, end=t*interval_ms)
            piano.notes.append(note)
            print("%f %f " % (note.start,note.end))
            k = -1 # Jump out of second loop
        k = k - 1
    t = t + 1
    
midi_obj.instruments.append(piano)
midi_obj.write('pretty_aria1.mid')

0.000000 0.952000 
0.960000 1.912000 
1.920000 2.032000 
2.040000 2.152000 
2.160000 2.632000 
2.640000 2.872000 
2.880000 3.352000 
3.360000 3.592000 
3.600000 3.832000 
3.840000 4.312000 
4.320000 5.760000 
5.764000 5.880000 
5.884000 6.004000 
6.008000 6.720000 
6.724000 6.844000 
6.848000 6.964000 
6.968000 7.084000 
7.088000 7.204000 
7.208000 7.324000 
7.328000 7.444000 
7.448000 7.564000 
7.568000 7.684000 
7.688000 7.804000 
7.808000 7.924000 
7.928000 8.044000 
8.048000 8.164000 
8.168000 8.404000 
8.408000 8.644000 
8.648000 8.752000 
8.760000 8.872000 
8.880000 9.112000 
9.120000 9.232000 
9.240000 9.364000 
9.368000 9.604000 
10.080000 11.512000 
11.520000 12.472000 
12.480000 13.432000 
13.440000 13.552000 
13.560000 13.672000 
13.680000 14.152000 
14.160000 14.392000 
14.400000 14.872000 
14.880000 15.112000 
15.120000 15.352000 
15.360000 15.832000 
15.840000 16.804000 
16.808000 16.912000 
16.920000 17.032000 
17.040000 17.152000 
17.160000 17.272000 
17.280000 17.39200

In [42]:
piano_roll.shape

(128, 33840)